In [113]:
import pandas as pd
data = pd.DataFrame(columns = ['price', 'street_add', 'latitude', 'longitude'])

In [186]:
from bs4 import BeautifulSoup
import requests
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# CHANGE ME
ZILLOW_LINK = "https://www.zillow.com/vancouver-bc/rentals/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-123.14039969240345%2C%22east%22%3A-123.12596940790333%2C%22south%22%3A49.2866690171266%2C%22north%22%3A49.29067179597184%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A791534%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A18%7D"
GOOGLE_FORM = "https://docs.google.com/forms/d/e/1FAIpQLSfaKlQyBZxSh48VDRlh_VTO_x6zHKJy6ZRPxkFohCrfhYgpwA/viewform?usp=sf_link"

if ZILLOW_LINK == "" or GOOGLE_FORM == "":
    print("Please update ZILLOW_LINK and GOOGLE_FORM in main.py!")
    exit(1)

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:91.0) Gecko/20100101 Firefox/91.0"
}

soup = BeautifulSoup(requests.get(ZILLOW_LINK, headers=headers).content, "html.parser")
ul_element = soup.find('ul', class_=["photo-cards"])
#print(ul_element.contents[0])


# uncomment this to print all data:
# with open('out.txt', 'w') as f:
#     f.write(json.dumps(data, indent=4))
#json.dumps(data, indent=4)

#driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for element in ul_element:
    try:
        print("-------------------------------------------------------------")
        #print(element.prettify())
        price_str = element.find('span', attrs={"data-test": "property-card-price"}).text.strip()
        price = ''.join(filter(lambda x: x.isdigit(), price_str))
        print(price)
        if price == "":
            continue
        script_tag = element.find('script', type="application/ld+json")
        json_data = json.loads(script_tag.string)
        print(json_data)
        street_add = json_data['address']['streetAddress']
        latitude = json_data['geo']['latitude']
        longitude = json_data['geo']['longitude']
        print(street_add)
        print(latitude)
        print(longitude)
        new_data = pd.DataFrame({'price': [price], 'street_add': [street_add], 'latitude': [latitude], 'longitude': [longitude]})
        data = pd.concat([new_data, data], ignore_index=True)
    except:
        pass


-------------------------------------------------------------
3439
{'@type': 'SingleFamilyResidence', '@context': 'http://schema.org', 'name': '825 Nicola St, Vancouver, BC V6G 2C4', 'floorSize': {'@type': 'QuantitativeValue', '@context': 'http://schema.org', 'value': '511'}, 'address': {'@type': 'PostalAddress', '@context': 'http://schema.org', 'streetAddress': '825 Nicola St', 'addressLocality': 'Vancouver', 'addressRegion': 'BC', 'postalCode': 'V6G2C4'}, 'geo': {'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 49.2886, 'longitude': -123.1316}, 'url': 'https://www.zillow.com/homedetails/825-Nicola-St-Vancouver-BC-V6G-2C4/2055456626_zpid/'}
825 Nicola St
49.2886
-123.1316
-------------------------------------------------------------
7000
{'@type': 'SingleFamilyResidence', '@context': 'http://schema.org', 'name': '1200 Georgia St W #3502, Vancouver, BC V6E 4R2', 'floorSize': {'@type': 'QuantitativeValue', '@context': 'http://schema.org', 'value': '1,200'}, 'addre

In [187]:
print(data)

    price                   street_add   latitude   longitude
0    4500         1288 Alberni St #502  49.286870 -123.126740
1    3950              1460 Barclay St  49.286930 -123.133320
2    7000      1200 Georgia St W #3502  49.287373 -123.126335
3    3439                825 Nicola St  49.288600 -123.131600
4    4500         1288 Alberni St #502  49.286870 -123.126740
5    3950              1460 Barclay St  49.286930 -123.133320
6    7000      1200 Georgia St W #3502  49.287373 -123.126335
7    3439                825 Nicola St  49.288600 -123.131600
8    3723               1005 Jervis St  49.284960 -123.132450
9    2875         1461 Harwood St #201  49.282940 -123.139050
10   3280       1050 Broughton St #311  49.284890 -123.133280
11   2200           1250 Comox St #801  49.283577 -123.132410
12   4900            1930 Yew St #1930  49.268790 -123.154976
13   4590      980 Cooperage Way #1207  49.273537 -123.114100
14   2690          2375 Broadway W #19  49.264230 -123.159260
15   300

In [188]:
data.drop_duplicates(subset=['street_add','price'], inplace=True)

In [172]:
data.reset_index(drop=True, inplace=True)

In [189]:
data = data[['price','street_add','latitude','longitude']]

In [190]:
data

,price,street_add,latitude,longitude
0,4500,1288 Alberni St #502,49.286870,-123.126740
1,3950,1460 Barclay St,49.286930,-123.133320
2,7000,1200 Georgia St W #3502,49.287373,-123.126335
3,3439,825 Nicola St,49.288600,-123.131600
8,3723,1005 Jervis St,49.284960,-123.132450
9,2875,1461 Harwood St #201,49.282940,-123.139050
10,3280,1050 Broughton St #311,49.284890,-123.133280
11,2200,1250 Comox St #801,49.283577,-123.132410
12,4900,1930 Yew St #1930,49.268790,-123.154976
13,4590,980 Cooperage Way #1207,49.273537,-123.114100


In [191]:
data.to_csv('zillow_rent.csv')